In [2]:
import tkinter as tk
import random
import math
import periodictable

class Ball:
    def __init__(self, size, color, velocity):
        self.size = size
        self.color = color
        self.velocity = velocity
        self.x = random.randint(size, 400-size)
        self.y = random.randint(size, 400-size)

 

    def draw(self, canvas):
        canvas.create_oval(self.x-self.size, self.y-self.size, self.x+self.size, self.y+self.size, fill=self.color)

 

    def move(self, balls):
        self.x += self.velocity[0]
        self.y += self.velocity[1]
        if self.x - self.size < 0 or self.x + self.size > 400:
            self.velocity[0] = -self.velocity[0]
        if self.y - self.size < 0 or self.y + self.size > 400:
            self.velocity[1] = -self.velocity[1]
        for ball in balls:
            if ball != self:
                distance = math.sqrt((self.x - ball.x)**2 + (self.y - ball.y)**2)
                if distance < self.size + ball.size:
                    # calculate the average velocity of the colliding balls
                    avg_velocity = [(self.velocity[0] + ball.velocity[0]) / 2,
                                    (self.velocity[1] + ball.velocity[1]) / 2]
                    # set the final velocity of both balls to the average velocity
                    self.velocity = avg_velocity
                    ball.velocity = avg_velocity


def check_collisions(balls):
    for i in range(len(balls)):
        for j in range(i+1, len(balls)):
            distance = math.sqrt((balls[i].x - balls[j].x)**2 + (balls[i].y - balls[j].y)**2)
            if distance < balls[i].size + balls[j].size:
                balls[i].velocity[0] = -balls[i].velocity[0]
                balls[i].velocity[1] = -balls[i].velocity[1]
                balls[j].velocity[0] = -balls[j].velocity[0]
                balls[j].velocity[1] = -balls[j].velocity[1]


# create a window and canvas
window = tk.Tk()
canvas = tk.Canvas(window, width=400, height=400) # Cria os limites da tela
canvas.pack()

resposta = "s" # Quando 'resposta' é igual a "s", a pergunta sobre qual elemento deve ser inserida é feita e,
               # por isso, possui esse valor inicial
    
balls = [] # Lista que armazena todas as bolas (átomos)

CORES = ['red', 'blue', 'green', 'yellow', 'orange', 'purple', 'pink'] # Lista com as cores que as bolas podem
                                                                       # assumir

while(resposta == "s"): # Enquanto resposta for sim, o sistema irá continuar pedindo para você informar elementos
                        # a serem inseridos
        
    simbolo_elemento = input("Qual elemento deseja inserir no sistema? Insira apenas o símbolo.")
    quantidade = int(input("Quantos desse elemento deseja inserir no sistema?"))
    
    element = getattr(periodictable, simbolo_elemento.capitalize()) # Busca o elemento no banco de dados da biblioteca
    raio = int(element.covalent_radius*10) # O raio atômico é multiplicado por 10 para obter tamanho proporcional à
                                           # tela e depois é convertido em inteiro
        
    cor = random.choice(CORES) # Escolhe aleatoriamente uma cor presente na lista 'CORES' 
    CORES.remove(cor) # Faz com que a mesma cor não seja sorteada para bolas diferentes
    
    for _ in range(quantidade): # O loop cria várias bolas do mesmo tipo, mas com velocidades diferentes
        velocidade = [random.randint(-100, 100), random.randint(-100, 100)] # A velocidade é uma lista em que a posição
                                                                            # 0 e 1 representam os eixos X e Y, respec-
                                                                            # tivamente. Cada valor é escolhido aleato-
                                                                            # riamente.
                    
        balls.append(Ball(raio, cor, velocidade)) # Adiciona a nova bola criada à lista
    
    resposta = input("Deseja inserir átomos de outro elemento? Responda com s ou n.").lower() # Caso a resposta seja 
                                                                            # diferente de sim, a simulação é iniciada

# draw the balls on the canvas
for ball in balls:
    ball.draw(canvas)

# move the balls every 50 milliseconds
def move_balls():
    for ball in balls:
        ball.move(balls)
        canvas.delete('all')
        for ball in balls:
            ball.draw(canvas)
    check_collisions(balls)
    canvas.after(50, move_balls)

move_balls()

# start the tkinter event loop
window.mainloop()

Qual elemento deseja inserir no sistema? Insira apenas o símbolo. o
Quantos desse elemento deseja inserir no sistema? 10
Deseja inserir átomos de outro elemento? Responda com s ou n. s
Qual elemento deseja inserir no sistema? Insira apenas o símbolo. h
Quantos desse elemento deseja inserir no sistema? 5
Deseja inserir átomos de outro elemento? Responda com s ou n. n
